In [1]:
from langchain_community.llms import Ollama

In [2]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
import nltk
import csv
import csv


In [7]:
import requests
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def query_ollama(prompt, model="FTQwen7BI_PIC24"):
    # Ollama API endpoint
    url = "http://localhost:11434/api/generate"
    
    # Payload for the API request
    payload = {
        "model": model, 
        "prompt": prompt,
        "stream": False  
    }
    
    response = requests.post(url, json=payload)
    
    if response.status_code == 200:
        return response.json()["response"]
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def calculate_bleu(candidate, reference):
    
    candidate_tokens = candidate.split()
    reference_tokens = [reference.split()]
    smoother = SmoothingFunction().method1

    return sentence_bleu(reference_tokens, candidate_tokens,smoothing_function=smoother)

question = "How do you configure a GPIO pin as an input in the PIC24 microcontroller?"
reference_answer = "To configure a GPIO pin as an input in the PIC24 microcontroller, set the corresponding bit in the TRISx register to '1'. For example, for RB0, use `TRISBbits.TRISB0 = 1;`."

generated_response = query_ollama(question)
print("Generated Response:", generated_response)

if generated_response:
    bleu_score = calculate_bleu(generated_response, reference_answer)
    print(f"BLEU Score: {bleu_score}")

Generated Response: To configure a GPIO pin as an input, set the corresponding bit in the TRIS register to '1'. For example: `TRISAbits.TRISA0 = 1;`.

BLEU Score: 0.4323200223374739


In [9]:
import csv
from nltk.translate.bleu_score import SmoothingFunction

test_set = []
with open('D:/Dev1/AI/FineTuneForPIC/data/test.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        test_set.append(row)  

results = []
for question, reference_answer in test_set:


    generated_response = query_ollama(question)
    if generated_response:
        bleu_score = calculate_bleu(generated_response, reference_answer)
        results.append((question, generated_response, bleu_score,reference_answer))
    else:
        results.append((question, "No response generated", 0.0))

for question, response, score, reference_answer in results:
    print(f"Question: {question}")
    print(f"Reference Response: {reference_answer}")
    print(f"Generated Response: {response}")
    print(f"BLEU Score: {score}\n_____________________________________________")

Question: instruction
Reference Response: response
Generated Response: response

BLEU Score: 0.1778279410038923
_____________________________________________
Question: What is the difference between a microcontroller (uC) and a microprocessor (uP)?
Reference Response: A microcontroller (uC) is designed as a single-chip solution with on-chip non-volatile memory for program storage, more interface functions (e.g., serial interfaces, timers, ADC), and does not support virtual memory. A microprocessor (uP) requires external support chips (e.g., memory, interface), lacks on-chip program memory, and supports virtual memory, making it suitable for running operating systems like Linux. uPs generally have higher performance in terms of clock speed, data width, and instruction set.
Generated Response: A microcontroller (uC) typically includes memory, interfaces, input/output (I/O), and other peripherals on-chip, while a microprocessor (uP) usually does not have these features. Both uCs and uPs u